<a href="https://www.kaggle.com/code/larsmagnusson/itf31519-feed-forward-networks?scriptVersionId=152691821" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/life-expectancy-who/Life Expectancy Data.csv
/kaggle/input/heart-failure-prediction/heart.csv


In [2]:
# Load our dataset
dataset = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')
dataset.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
# Separate features from targets
features = dataset.drop('HeartDisease',axis=1)
targets = dataset['HeartDisease']

# Encode our targets using one-hot encoding
targets_onehot = pd.get_dummies(targets)
targets_onehot.head()

,0,1
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0


In [4]:
# Check distribution of target values
targets.value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [5]:
# One-hot encode categorical features
features_onehot = pd.get_dummies(features)
features_onehot.shape

(918, 20)

In [6]:
# Convert to float (probably not needed, but neural networks use floats 
# internally, so this lets us specify the type explicitly)
features_onehot = features_onehot.astype('float32')
targets_onehot = targets_onehot.astype('float32')
features_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    float32
 1   RestingBP          918 non-null    float32
 2   Cholesterol        918 non-null    float32
 3   FastingBS          918 non-null    float32
 4   MaxHR              918 non-null    float32
 5   Oldpeak            918 non-null    float32
 6   Sex_F              918 non-null    float32
 7   Sex_M              918 non-null    float32
 8   ChestPainType_ASY  918 non-null    float32
 9   ChestPainType_ATA  918 non-null    float32
 10  ChestPainType_NAP  918 non-null    float32
 11  ChestPainType_TA   918 non-null    float32
 12  RestingECG_LVH     918 non-null    float32
 13  RestingECG_Normal  918 non-null    float32
 14  RestingECG_ST      918 non-null    float32
 15  ExerciseAngina_N   918 non-null    float32
 16  ExerciseAngina_Y   918 non

In [7]:
# Split our data into 90/10
from sklearn.model_selection import train_test_split
train_features, test_features, train_targets, test_targets = train_test_split(features_onehot, targets_onehot, test_size=0.1, stratify=targets)

In [8]:
import tensorflow as tf

# Convert the dataframes into tensors
train_features_tensor = tf.convert_to_tensor(train_features)
train_targets_tensor = tf.convert_to_tensor(train_targets)
test_features_tensor = tf.convert_to_tensor(test_features)
test_targets_tensor = tf.convert_to_tensor(test_targets)

test_targets.shape

(92, 2)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create our neural network. 
model = Sequential([Dense(20,activation='sigmoid'), Dense(2,activation="sigmoid")])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
model.fit(train_features, train_targets, epochs=20, batch_size=2)

Epoch 1/20
413/413 [==============================] - 1s 1ms/step - loss: 0.7146 - accuracy: 0.5908
Epoch 2/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6054 - accuracy: 0.6804
Epoch 3/20
413/413 [==============================] - 1s 1ms/step - loss: 0.5782 - accuracy: 0.6998
Epoch 4/20
413/413 [==============================] - 1s 1ms/step - loss: 0.5456 - accuracy: 0.7470
Epoch 5/20
413/413 [==============================] - 1s 1ms/step - loss: 0.5255 - accuracy: 0.7627
Epoch 6/20
413/413 [==============================] - 1s 1ms/step - loss: 0.5098 - accuracy: 0.7724
Epoch 7/20
413/413 [==============================] - 1s 1ms/step - loss: 0.4959 - accuracy: 0.7797
Epoch 8/20
413/413 [==============================] - 1s 1ms/step - loss: 0.4762 - accuracy: 0.8027
Epoch 9/20
413/413 [==============================] - 1s 1ms/step - loss: 0.4550 - accuracy: 0.8123
Epoch 10/20
413/413 [==============================] - 1s 1ms/step - loss: 0.4460 - accuracy: 0.8087

In [11]:
model.evaluate(test_features, test_targets)

3/3 [==============================] - 0s 3ms/step - loss: 0.3252 - accuracy: 0.8804


[0.32518136501312256, 0.8804348111152649]

In [12]:
model_simple = Sequential([Dense(10,activation='sigmoid'), Dense(2,activation="sigmoid")])
model_simple.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_simple.fit(train_features, train_targets, epochs=20, batch_size=2)

Epoch 1/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6639 - accuracy: 0.5375
Epoch 2/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6460 - accuracy: 0.5617
Epoch 3/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6430 - accuracy: 0.5726
Epoch 4/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6397 - accuracy: 0.5557
Epoch 5/20
413/413 [==============================] - 1s 2ms/step - loss: 0.6376 - accuracy: 0.5787
Epoch 6/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6383 - accuracy: 0.5617
Epoch 7/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6386 - accuracy: 0.5605
Epoch 8/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6382 - accuracy: 0.5630
Epoch 9/20
413/413 [==============================] - 1s 1ms/step - loss: 0.6371 - accuracy: 0.5860
Epoch 10/20
413/413 [==============================] - 1s 2ms/step - loss: 0.6359 - accuracy: 0.5642

In [13]:
model_simple.evaluate(test_features, test_targets)

3/3 [==============================] - 0s 4ms/step - loss: 0.5880 - accuracy: 0.6522


[0.5880406498908997, 0.6521739363670349]

In [14]:
# Load and Pre-process the dataset. This code has been copied from the workshop on Regression for Life expectancy

# Load the dataset
dataset2 = pd.read_csv('/kaggle/input/life-expectancy-who/Life Expectancy Data.csv')
dataset2.head()

# Drop all rows where we are missing the life expectancy. Had to put the key in a list 
# for this to work in this notebook!?
dataset2 = dataset2.dropna(subset=['Life expectancy '])

# Fill the remaining missing values with the means for each column
float_cols_with_nas = ['Alcohol','Hepatitis B',' BMI ','Polio','Total expenditure','Diphtheria ','GDP','Population',' thinness  1-19 years',' thinness 5-9 years','Income composition of resources','Schooling']
dataset2[float_cols_with_nas] = dataset2[float_cols_with_nas].fillna(dataset2[float_cols_with_nas].mean())

# Convert the categorical columns to appropriate types
dataset2[['Country', 'Status']] = dataset2[['Country', 'Status']].astype('category')

# Encode the categorical columns using one-hot encoding
dataset2 = pd.get_dummies(dataset2)

# Separate into features/targets
targets2 = dataset2['Life expectancy ']
features2 = dataset2.drop('Life expectancy ', axis=1)

from sklearn.model_selection import train_test_split
# Split the data into 75/25 (as specified in the assignment)
train_targets2, test_targets2, train_features2, test_features2 = train_test_split(targets2,features2)

features2.head()

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,...,Country_Uruguay,Country_Uzbekistan,Country_Vanuatu,Country_Venezuela (Bolivarian Republic of),Country_Viet Nam,Country_Yemen,Country_Zambia,Country_Zimbabwe,Status_Developed,Status_Developing
0,2015,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,...,0,0,0,0,0,0,0,0,0,1
1,2014,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,...,0,0,0,0,0,0,0,0,0,1
2,2013,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,...,0,0,0,0,0,0,0,0,0,1
3,2012,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,...,0,0,0,0,0,0,0,0,0,1
4,2011,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
model2 = Sequential([Dense(units=204,activation='sigmoid'), Dense(units=1,activation="linear")])
model2.compile(optimizer='adam',loss='mean_squared_error',metrics=['mean_absolute_error'])
model2.fit(train_features2, train_targets2, epochs=20, batch_size=1)

Epoch 1/20
2196/2196 [==============================] - 4s 2ms/step - loss: 711.8493 - mean_absolute_error: 18.8512
Epoch 2/20
2196/2196 [==============================] - 4s 2ms/step - loss: 90.9433 - mean_absolute_error: 7.7418
Epoch 3/20
2196/2196 [==============================] - 4s 2ms/step - loss: 88.5904 - mean_absolute_error: 7.6665
Epoch 4/20
2196/2196 [==============================] - 4s 2ms/step - loss: 87.8466 - mean_absolute_error: 7.6425
Epoch 5/20
2196/2196 [==============================] - 4s 2ms/step - loss: 87.4744 - mean_absolute_error: 7.6290
Epoch 6/20
2196/2196 [==============================] - 4s 2ms/step - loss: 86.9037 - mean_absolute_error: 7.5938
Epoch 7/20
2196/2196 [==============================] - 4s 2ms/step - loss: 86.9353 - mean_absolute_error: 7.5795
Epoch 8/20
2196/2196 [==============================] - 4s 2ms/step - loss: 87.0381 - mean_absolute_error: 7.5880
Epoch 9/20
2196/2196 [==============================] - 4s 2ms/step - loss: 86.6617 - 

In [16]:
model2.evaluate(test_features2, test_targets2)
model2(test_features2[:20].to_numpy())

23/23 [==============================] - 0s 2ms/step - loss: 93.9469 - mean_absolute_error: 7.7301


<tf.Tensor: shape=(20, 1), dtype=float32, numpy=
array([[69.97194 ],
       [69.97194 ],
       [69.97194 ],
       [74.23651 ],
       [69.97194 ],
       [69.97194 ],
       [69.97194 ],
       [69.97194 ],
       [69.97194 ],
       [69.97194 ],
       [69.97194 ],
       [74.729195],
       [69.97194 ],
       [69.97194 ],
       [73.51901 ],
       [69.97194 ],
       [69.97194 ],
       [73.005806],
       [70.63792 ],
       [69.97194 ]], dtype=float32)>

In [17]:
from tensorflow.keras.layers import Normalization

normalization = Normalization(axis=-1)
normalization.adapt(train_features2)
model3 = Sequential([normalization,Dense(units=40,activation='sigmoid'),Dense(units=1)])
model3.compile(optimizer='sgd',loss='mean_squared_error',metrics=['mean_absolute_error'])
model3.fit(train_features2, train_targets2, epochs=20, batch_size=24)


Epoch 1/20
92/92 [==============================] - 1s 2ms/step - loss: 150.3060 - mean_absolute_error: 5.5822
Epoch 2/20
92/92 [==============================] - 0s 2ms/step - loss: 5.1676 - mean_absolute_error: 1.5073
Epoch 3/20
92/92 [==============================] - 0s 2ms/step - loss: 4.2927 - mean_absolute_error: 1.3406
Epoch 4/20
92/92 [==============================] - 0s 2ms/step - loss: 3.7671 - mean_absolute_error: 1.2478
Epoch 5/20
92/92 [==============================] - 0s 2ms/step - loss: 3.3332 - mean_absolute_error: 1.1557
Epoch 6/20
92/92 [==============================] - 0s 2ms/step - loss: 3.1414 - mean_absolute_error: 1.1317
Epoch 7/20
92/92 [==============================] - 0s 2ms/step - loss: 2.9341 - mean_absolute_error: 1.0904
Epoch 8/20
92/92 [==============================] - 0s 2ms/step - loss: 2.7820 - mean_absolute_error: 1.0514
Epoch 9/20
92/92 [==============================] - 0s 2ms/step - loss: 2.5193 - mean_absolute_error: 1.0010
Epoch 10/20
92/92

In [18]:
model3.evaluate(test_features2, test_targets2)
model3(test_features2[:40])

23/23 [==============================] - 0s 2ms/step - loss: 4.6353 - mean_absolute_error: 1.2102


<tf.Tensor: shape=(40, 1), dtype=float32, numpy=
array([[75.264656],
       [75.11153 ],
       [64.55697 ],
       [74.306915],
       [75.82763 ],
       [73.27877 ],
       [59.86589 ],
       [44.60534 ],
       [75.60998 ],
       [71.69901 ],
       [82.263855],
       [73.994736],
       [50.45683 ],
       [82.55584 ],
       [80.39854 ],
       [59.07518 ],
       [73.19376 ],
       [72.71873 ],
       [65.87103 ],
       [79.66889 ],
       [80.94123 ],
       [72.73457 ],
       [73.34868 ],
       [71.3385  ],
       [81.78663 ],
       [61.06107 ],
       [81.7591  ],
       [64.03714 ],
       [83.86419 ],
       [63.511097],
       [69.82775 ],
       [50.884045],
       [73.583336],
       [74.25579 ],
       [75.21583 ],
       [72.589226],
       [74.42945 ],
       [74.401436],
       [74.554825],
       [76.57916 ]], dtype=float32)>

In [19]:
model4 = Sequential()
model4.add(normalization)
model4.add(Dense(40, activation='relu'))
model4.add(Dense(40, activation='relu'))
model4.add(Dense(40, activation='relu'))
model4.add(Dense(40, activation='relu'))
model4.add(Dense(40, activation='relu'))
model4.add(Dense(1, activation='linear'))
print(model4.summary())
model4.compile('adam',loss='mean_squared_error',metrics=['mean_absolute_error'])

model4.fit(train_features2,train_targets2, epochs=40, batch_size=20)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 204)               409       
_________________________________________________________________
dense_8 (Dense)              (None, 40)                8200      
_________________________________________________________________
dense_9 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_10 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_11 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_12 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                

In [20]:
import keras
inputs = keras.Input(shape=(204,))
print(inputs.shape)
norm_inputs = normalization(inputs)
x = Dense(40, activation='relu')(norm_inputs)
x = Dense(40, activation='relu')(x)
x = Dense(40, activation='relu')(x)
x = Dense(40, activation='relu')(x)
x = Dense(40, activation='relu')(x) + Dense(40,activation='relu')(norm_inputs)
x = Dense(1, activation='linear')(x)
model5 = keras.Model(inputs=inputs, outputs=x)
print(model5.summary())
model5.compile('adam',loss='mean_squared_error',metrics=['mean_absolute_error'])

model5.fit(train_features2,train_targets2, epochs=20, batch_size=20)


(None, 204)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 204)]        0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 204)          409         input_1[0][0]                    
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 40)           8200        normalization[2][0]              
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 40)           1640        dense_14[0][0]                   
__________________________________________________________________________________